In [2]:
import torch

# 处理图像

In [4]:
import imageio  # can also use torchvision

In [ ]:
img_arr = imageio.v3.imread('./image-dog/bobby.jpg')
print(img_arr.shape)
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)
print(out.shape)

In [6]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

import os
data_dir = './image-cats/'
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == '.png']
for i, filename in enumerate(filenames):
    img_arr = imageio.v3.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3]
    batch[i] = img_t    # 把img_t输入batch
batch = batch.float()   # Tensor要化成浮点
# 神经网络输入数据在0~1或-1~1范围内效果最佳
batch /= 255.0